In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pdb
import os

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from pandas_summary import DataFrameSummary

from fastai.model import *
from fastai.dataset import *
from fastai.lm_rnn import *
from fastai.sgdr import *
from fastai.rnn_reg import EmbeddingDropout, WeightDrop, LockedDropout
from fastai.torch_imports import *

import torchtext
from torchtext import vocab, data

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import spacy
from spacy.lang.en.stop_words import STOP_WORDS as spacy_STOPWORDS
spacy_en = spacy.load('en')

from wordcloud import WordCloud, STOPWORDS

# pandas and plotting config
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [3]:
PATH = 'data'

os.makedirs(f'{PATH}/models', exist_ok=True)
os.makedirs(f'{PATH}/tmp', exist_ok=True)
os.makedirs(f'{PATH}/submissions', exist_ok=True)

Load data and define labels (ordering is important for competition submission!)

*Note: We are also adding a "None" column

In [4]:
raw_train_df = pd.read_csv(f'{PATH}/train_preproc.csv')
test_df = pd.read_csv(f'{PATH}/test_preproc.csv')
sample_subm_df = pd.read_csv(f'{PATH}/sample_submission.csv')

txt_col = 'comment_text_cleaned'

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
raw_train_df['none'] = 1 - raw_train_df[label_cols].max(axis=1)

model_cols = ['id', txt_col] + label_cols + ['none']

In [5]:
n_folds = 10
n_trn = len(raw_train_df)
n_examples_per_fold = n_trn // n_folds

# n_examples_per_fold

## Prepare Data

*Note: Only need to run 1x (or as desired to regenerate these .csv files)

Build cross-validation datasets

In [6]:
raw_train_df_rand = raw_train_df.sample(frac=1, random_state=9) # frac=1 = return all rows in random order

In [7]:
val_dfs = []

for i in range(0, n_folds):
    start = i * n_examples_per_fold
    end = n_examples_per_fold + start if (i + 1 < n_folds) else None
    val_dfs.append(raw_train_df_rand[start:end])
    
# [ print(idx,len(d)) for idx, d in enumerate(val_dfs) ]

In [8]:
trn_dfs = []

for idx, df in enumerate(val_dfs):
    trn_dfs.append(pd.concat([ val_df for val_idx, val_df in enumerate(val_dfs) if val_idx != idx]))
    
# [ print(idx,len(d)) for idx, d in enumerate(trn_dfs) ]

In [9]:
for idx, [trn_df, val_df] in enumerate(zip(trn_dfs, val_dfs)):
    print(idx, len(trn_df), len(val_df))
    
    trn_df[model_cols].to_csv(f'{PATH}/train_ds_{idx}_of_{n_folds}.csv', index=None)
    val_df[model_cols].to_csv(f'{PATH}/valid_ds_{idx}_of_{n_folds}.csv', index=None)

0 143614 15957
1 143614 15957
2 143614 15957
3 143614 15957
4 143614 15957
5 143614 15957
6 143614 15957
7 143614 15957
8 143614 15957
9 143613 15958


Use the below if you want to create a single training and cv dataset

In [10]:
# split the training data into a train and validatin dataset
trn, val = train_test_split(raw_train_df, test_size=0.05, random_state=9)
print(len(trn), len(val), len(trn[trn.none != 1]), len(val[val.none != 1]))

# save train, val, and test datasets for torchtext
trn[model_cols].to_csv(f'{PATH}/train_ds.csv', index=None)
val[model_cols].to_csv(f'{PATH}/valid_ds.csv', index=None)

# save full cleaned datasets (train+valid and test) as well
raw_train_df[model_cols].to_csv(f'{PATH}/full_train_ds.csv', index=None)
test_df[['id', txt_col]].to_csv(f'{PATH}/test_ds.csv', index=None)

151592 7979 15417 808


In [11]:
display(pd.read_csv("data/full_train_ds.csv").head(2))
display(pd.read_csv("data/test_ds.csv").head(2))

,id,comment_text_cleaned,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
0,0000997932d777bf,"explanation why the edits made under my username hardcore metallica fan were reverted? they were not vandalisms, just closure on some gas after i voted at new york dolls fac. and please do not remove the template from the talk page since i am retired now.",0,0,0,0,0,0,1
1,000103f0d9cfb60f,"d'aww! he matches this background colour i am seemingly stuck with. thanks. (talk) 21:51, january 11, 2016 (utc)",0,0,0,0,0,0,1


,id,comment_text_cleaned
0,00001cee341fdb12,"yo bitch ja rule is more succesful then you will ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"
1,0000247867823ef7,"== from rfc == the title is fine as it is, imo."


## Build Datasets and DataLoaders

Define hyperparameters and column that holds text data

In [6]:
max_features = 100000 #30000
min_freq = 10 #0
max_len = 175 #100

pretrained_vectors = None #'fasttext.en.300d'

batch_sizes = (64,64,64)

Configure how we are going to process text and label fields

In [7]:
import re, string

re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [8]:
TEXT_fld = data.Field(sequential=True, tokenize=tokenize, lower=True, fix_length=max_len)
LABEL_fld = data.Field(sequential=False, use_vocab=False, tensor_type=torch.cuda.ByteTensor)

There are various built-in Datasets in torchtext that handle common use cases. **For csv/tsv files, the TabularDataset class** is convenient. Here’s how we would read data from a csv file using the TabularDataset:

In [9]:
%%time

# train/validation
train_datafields = [("id", None), # we won't be needing the id, so we pass in None as the field
                    (txt_col, TEXT_fld),
                    ("toxic", LABEL_fld), ("severe_toxic", LABEL_fld), ("obscene", LABEL_fld),
                    ("threat", LABEL_fld), ("insult", LABEL_fld), ("identity_hate", LABEL_fld), ("none", None)]

train_ds, valid_ds = data.TabularDataset.splits(PATH, train='train_ds.csv', validation='valid_ds.csv',
                                          format='csv', skip_header=True, fields=train_datafields)

# test
test_datafields = [("id", None), (txt_col, TEXT_fld)]

test_ds = data.TabularDataset(f'{PATH}/test_ds.csv', format='csv', skip_header=True, fields=test_datafields)

# train+val
full_train_ds = data.TabularDataset(f'{PATH}/full_train_ds.csv', 
                                    format='csv', skip_header=True, fields=train_datafields)

CPU times: user 40.4 s, sys: 1.32 s, total: 41.7 s
Wall time: 41.8 s


In [10]:
display(train_ds[0])
display(train_ds[0].__dict__.keys())
display(train_ds[1].comment_text_cleaned[:5])

dict_keys(['comment_text_cleaned', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

['you', 'cunt']

Build vocab on *full* training data

In [11]:
TEXT_fld.build_vocab(full_train_ds, min_freq=min_freq, max_size=max_features, vectors=pretrained_vectors)

In [12]:
# The vocab.freqs is a collections.Counter object, so we can take a look at the most frequent words.
TEXT_fld.vocab.freqs.most_common(10)

[('.', 645337),
 ('the', 496748),
 (',', 473218),
 ('"', 392156),
 ('to', 297318),
 ('i', 240305),
 ('of', 224837),
 ('and', 224115),
 ('is', 222448),
 ('you', 221861)]

In [13]:
train_iter, val_iter = data.BucketIterator.splits(
    (train_ds, valid_ds), # we pass in the datasets we want the iterator to draw data from
    batch_sizes=(batch_sizes[0], batch_sizes[1]),
    device=0, # if you want to use the GPU, specify the GPU number here
    sort_key=lambda x: len(x.comment_text_cleaned), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False,
    repeat=False # we pass repeat=False because we want to wrap this Iterator layer.
)

In [14]:
batch = next(train_iter.__iter__()); 

display(batch)
display(batch.__dict__.keys())

dict_keys(['batch_size', 'dataset', 'train', 'comment_text_cleaned', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

For the test set, we don't want the data to be shuffled. This is why we'll be using a standard Iterator.

In [15]:
test_iter = data.Iterator(test_ds, batch_size=batch_sizes[2], device=0, train=False, 
                          shuffle=False, sort=False, sort_within_batch=False, repeat=False)

In [16]:
class BatchWrapper:
    def __init__(self, dl, x_var, y_vars):
        self.dl, self.x_var, self.y_vars = dl, x_var, y_vars # we pass in the list of attributes for x and y
    
    def __iter__(self):
        for batch in self.dl:
            x = getattr(batch, self.x_var) # we assume only one input in this wrapper
            
            if self.y_vars is not None: # we will concatenate y into a single tensor
                y = torch.cat([ getattr(batch, feat).unsqueeze(1) for feat in self.y_vars ], dim=1).float()
            else:
                y = torch.zeros((1))

            yield (x, y)
    
    def __len__(self):
        return len(self.dl)

In [17]:
train_dl = BatchWrapper(train_iter, txt_col, label_cols)
valid_dl = BatchWrapper(val_iter, txt_col, label_cols)
test_dl = BatchWrapper(test_iter, txt_col, None)

Construct a fastai ModelData

In [18]:
md = ModelData(PATH, trn_dl=train_dl, val_dl=valid_dl, test_dl=test_dl)

## Define the Model

Define a simple GRU

In [19]:
class SimpleGru(nn.Module):
    def __init__(self, vocab_sz, emb_sz=300, n_rnn_hidden=256, n_rnn_layers=1, bi_dir=True, out_sz=1, bsz=64,
                 linears=[512], linear_drops=[0.1],
                 dropouth=0.3, dropouti=0.4, dropoute=0.1, wdrop=0.05, use_bn=False):
        
        super().__init__() 
        
        self.bsz = bsz
               
        # configure embeddings layer
        self.dropoute = dropoute
        self.dropouti = LockedDropout(dropouti)
        
        self.emb = nn.Embedding(vocab_sz, emb_sz)
        self.emb.data = train_ds.fields[txt_col].vocab.vectors # to use the pretrained vectors
        #self.emb.weight.requires_grad=False
        self.emb_with_drop = EmbeddingDropout(self.emb)
        
        # configure rnns
        self.n_rnn_hidden, self.n_rnn_layers, self.n_dirs = n_rnn_hidden, n_rnn_layers, 2 if bi_dir else 1
        self.rnn = nn.GRU(emb_sz, self.n_rnn_hidden, self.n_rnn_layers, bidirectional=bi_dir, dropout=dropouth)
        if wdrop: self.rnn = WeightDrop(self.rnn, wdrop)
      
        # configure optional FC layers (3 because we are concat 3 outputs of bsz*n_rnn_hidden into first FC);
        # if not concatentatin outp[-1], then make this 2
        linears = [n_rnn_hidden * 2 * self.n_dirs] + linears
        self.use_bn = use_bn
        
        self.linears = nn.ModuleList([
            nn.Linear(linears[idx], linears[idx + 1]) for idx in range(len(linears) - 1)
        ])
        self.linear_bns = nn.ModuleList([
            nn.BatchNorm1d(sz) for sz in linears[1:]
        ])
        self.linear_drops = nn.ModuleList([
            nn.Dropout(drop) for drop in linear_drops
        ])
        
        self.outp = nn.Linear(linears[-1], out_sz)
        
        # initialize weights
        for o in self.linears: kaiming_normal(o.weight.data)
        kaiming_normal(self.outp.weight.data)
        
        # init hidden
        self.init_hidden(self.bsz)
    
    def forward(self, seq):
        bsz = seq.size(1)
        if (self.hidden[0].size(1) != bsz): self.init_hidden(bsz)
        
        x = self.emb_with_drop(seq, dropout=self.dropoute if self.training else 0)
        x = self.dropouti(x)
        
        #pdb.set_trace()
        output, h = self.rnn(x, self.hidden)        
        self.hidden = repackage_var(h)
        
        sl, bs, _ = output.size()
  
        avg_pool = F.adaptive_avg_pool1d(output.permute(1,2,0), (1,)).view(bs,-1)   
        max_pool = F.adaptive_max_pool1d(output.permute(1,2,0), (1,)).view(bs,-1) 
        
        x = torch.cat([avg_pool, max_pool], dim=1)
        
        for l, d, b in zip (self.linears, self.linear_drops, self.linear_bns):
            x = F.relu(l(x))
            if(self.use_bn): x = b(x)
            x = d(x)
        
        outp = F.sigmoid(self.outp(x))
        return outp

    def init_hidden(self, bsz):
        self.hidden = V(torch.zeros(self.n_dirs * self.n_rnn_layers, bsz, self.n_rnn_hidden))

In [20]:
class SimpleLstm(nn.Module):
    def __init__(self, vocab_sz, emb_sz=300, n_rnn_hidden=256, n_rnn_layers=1, bi_dir=True, out_sz=1, bsz=64,
                 linears=[512], linear_drops=[0.1],
                 dropouth=0.3, dropouti=0.4, dropoute=0.1, wdrop=0.05, use_bn=False):
        
        super().__init__() 
        
        self.bsz = bsz
               
        # configure embeddings layer
        self.dropoute = dropoute
        self.dropouti = LockedDropout(dropouti)
        
        self.emb = nn.Embedding(vocab_sz, emb_sz)
        self.emb.data = train_ds.fields[txt_col].vocab.vectors # to use the pretrained vectors
        #self.emb.weight.requires_grad=False
        self.emb_with_drop = EmbeddingDropout(self.emb)
        
        # configure rnns
        self.n_rnn_hidden, self.n_rnn_layers, self.n_dirs = n_rnn_hidden, n_rnn_layers, 2 if bi_dir else 1
        self.rnn = nn.LSTM(emb_sz, self.n_rnn_hidden, self.n_rnn_layers, bidirectional=bi_dir, dropout=dropouth)
        if wdrop: self.rnn = WeightDrop(self.rnn, wdrop)
      
        # configure optional FC layers (3 because we are concat 3 outputs of bsz*n_rnn_hidden into first FC);
        # if not concatentatin outp[-1], then make this 2
        linears = [n_rnn_hidden * 2 * self.n_dirs] + linears
        self.use_bn = use_bn
        
        self.linears = nn.ModuleList([
            nn.Linear(linears[idx], linears[idx + 1]) for idx in range(len(linears) - 1)
        ])
        self.linear_bns = nn.ModuleList([
            nn.BatchNorm1d(sz) for sz in linears[1:]
        ])
        self.linear_drops = nn.ModuleList([
            nn.Dropout(drop) for drop in linear_drops
        ])
        
        self.outp = nn.Linear(linears[-1], out_sz)
        
        # initialize weights
        for o in self.linears: kaiming_normal(o.weight.data)
        kaiming_normal(self.outp.weight.data)
        
        # init hidden
        self.init_hidden(self.bsz)
    
    def forward(self, seq):
        bsz = seq.size(1)
        if (self.hidden[0].size(1) != bsz): self.init_hidden(bsz)
        
        x = self.emb_with_drop(seq, dropout=self.dropoute if self.training else 0)
        x = self.dropouti(x)
        
        #pdb.set_trace()
        output, h = self.rnn(x, self.hidden)        
        self.hidden = repackage_var(h)
        
        sl, bs, _ = output.size()
  
        avg_pool = F.adaptive_avg_pool1d(output.permute(1,2,0), (1,)).view(bs,-1)   
        max_pool = F.adaptive_max_pool1d(output.permute(1,2,0), (1,)).view(bs,-1) 
        
        x = torch.cat([avg_pool, max_pool], dim=1)
        
        for l, d, b in zip (self.linears, self.linear_drops, self.linear_bns):
            x = F.relu(l(x))
            if(self.use_bn): x = b(x)
            x = d(x)
        
        outp = F.sigmoid(self.outp(x))
        return outp

    def init_hidden(self, bsz):
        self.hidden = (V(torch.zeros(self.n_dirs * self.n_rnn_layers, bsz, self.n_rnn_hidden)),
                       V(torch.zeros(self.n_dirs * self.n_rnn_layers, bsz, self.n_rnn_hidden)))

In [25]:
vocab_sz = len(TEXT_fld.vocab)
emb_sz = 300

n_rnn_hidden = 128
n_rnn_layers = 1
bi_dir = True

out_sz = 6
linears = []; linear_drops = []; use_bn = False

model = SimpleLstm(vocab_sz, emb_sz, n_rnn_hidden, n_rnn_layers, True, out_sz, bsz=batch_sizes[0],
                             linears=linears, linear_drops=linear_drops, use_bn=use_bn)

model.cuda()

SimpleLstm(
  (dropouti): LockedDropout(
  )
  (emb): Embedding(26970, 300)
  (emb_with_drop): EmbeddingDropout(
    (embed): Embedding(26970, 300)
  )
  (rnn): WeightDrop(
    (module): LSTM(300, 128, dropout=0.3, bidirectional=True)
  )
  (linears): ModuleList(
  )
  (linear_bns): ModuleList(
  )
  (linear_drops): ModuleList(
  )
  (outp): Linear(in_features=512, out_features=6, bias=True)
)

In [82]:
class ConvLstm(nn.Module):
    def __init__(self, vocab_sz, emb_sz=300, n_rnn_hidden=256, n_rnn_layers=1, bi_dir=True, out_sz=1, bsz=64,
                 dropouth=0.3, dropouti=0.4, dropoute=0.1, wdrop=0.05):
        
        super().__init__() 
        
        self.bsz = bsz
               
        # configure embeddings layer
        self.dropoute = dropoute
        self.dropouti = LockedDropout(dropouti)
        
        self.emb = nn.Embedding(vocab_sz, emb_sz)
        self.emb.data = train_ds.fields[txt_col].vocab.vectors # to use the pretrained vectors
        #self.emb.weight.requires_grad=False
        self.emb_with_drop = EmbeddingDropout(self.emb)
        
        # configure rnns
        self.n_rnn_hidden, self.n_rnn_layers, self.n_dirs = n_rnn_hidden, n_rnn_layers, 2 if bi_dir else 1
        self.rnn = nn.LSTM(emb_sz, self.n_rnn_hidden, self.n_rnn_layers, bidirectional=bi_dir, dropout=dropouth)
        if wdrop: self.rnn = WeightDrop(self.rnn, wdrop)
            
        self.conv = nn.Conv1d(n_rnn_hidden * self.n_dirs, 64, 4)
      
        self.outp = nn.Linear(64 * 2, out_sz)
        
        # initialize weights
        kaiming_normal(self.outp.weight.data)
        
        # init hidden
        self.init_hidden(self.bsz)
    
    def forward(self, seq):
        bsz = seq.size(1)
        if (self.hidden[0].size(1) != bsz): self.init_hidden(bsz)
        
        x = self.emb_with_drop(seq, dropout=self.dropoute if self.training else 0)
        x = self.dropouti(x)
        
        #pdb.set_trace()
        output, h = self.rnn(x, self.hidden)        
        self.hidden = repackage_var(h)
     
        sl, bs, _ = output.size()
        x = F.relu(self.conv(output.permute(1,2,0)))
  
        avg_pool = F.adaptive_avg_pool1d(x, (1,)).view(bs,-1)   
        max_pool = F.adaptive_max_pool1d(x, (1,)).view(bs,-1) 
        
        x = torch.cat([avg_pool, max_pool], dim=1)

        outp = F.sigmoid(self.outp(x))
        return outp

    def init_hidden(self, bsz):
        self.hidden = (V(torch.zeros(self.n_dirs * self.n_rnn_layers, bsz, self.n_rnn_hidden)),
                       V(torch.zeros(self.n_dirs * self.n_rnn_layers, bsz, self.n_rnn_hidden)))

In [83]:
vocab_sz = len(TEXT_fld.vocab)
emb_sz = 300

n_rnn_hidden = 128
n_rnn_layers = 1
bi_dir = True

out_sz = 6

model = ConvLstm(vocab_sz, emb_sz, n_rnn_hidden, n_rnn_layers, True, out_sz, bsz=batch_sizes[0])

model.cuda()

ConvLstm(
  (dropouti): LockedDropout(
  )
  (emb): Embedding(26970, 300)
  (emb_with_drop): EmbeddingDropout(
    (embed): Embedding(26970, 300)
  )
  (rnn): WeightDrop(
    (module): LSTM(300, 128, dropout=0.3, bidirectional=True)
  )
  (conv): Conv1d(256, 64, kernel_size=(4,), stride=(1,))
  (outp): Linear(in_features=128, out_features=6, bias=True)
)

## Train Model

In [84]:
lo = LayerOptimizer(optim.Adam, model, 1e-2, 1e-5)

In [85]:
on_end = lambda sched, cycle: save_model(model, f'{PATH}/models/lstm_fit_1_cyc_{cycle}')

In [86]:
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]

In [87]:
fit(model, md, 2**4-1, lo.opt, F.binary_cross_entropy, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

  0%|          | 1/2369 [00:00<14:12,  2.78it/s, loss=0.752]

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/module.py:357: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  result = self.forward(*input, **kwargs)


/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  


epoch      trn_loss   val_loss   
    0      0.051143   0.045428  
    1      0.052452   0.048086                                  
    2      0.048935   0.042793                                  
    3      0.057762   0.053487                                  
    4      0.050881   0.046729                                  
    5      0.049764   0.043361                                  
    6      0.045766   0.041723                                  
    7      0.060439   0.052335                                  
    8      0.060282   0.052392                                  
    9      0.061376   0.049558                                  
    10     0.052035   0.045024                                  
    11     0.050743   0.045635                                  
    12     0.050397   0.042907                                  
    13     0.045974   0.042163                                  
    14     0.04428    0.042109                                  



[0.042109344]

In [97]:
model.load_state_dict(torch.load(f'{PATH}/models/lstm_fit_1_cyc_3'))

In [98]:
# cb[0].plot_lr()
# cb[0].plot_loss()

In [103]:
on_end = lambda sched, cycle: save_model(model, f'{PATH}/models/lstm_fit_2_cyc_{cycle}')

lo = LayerOptimizer(optim.Adam, model, 1e-2, 1e-5)

cb = [CosAnneal(lo, (len(md.trn_dl) * 20), on_cycle_end=on_end)]

fit(model, md, 20, lo.opt, F.binary_cross_entropy, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=20), HTML(value='')))

  0%|          | 1/2369 [00:00<12:26,  3.17it/s, loss=0.0313]

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/module.py:357: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  result = self.forward(*input, **kwargs)


/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  


epoch      trn_loss   val_loss   
    0      0.047446   0.042326  
    1      0.060109   0.052738                                  
    2      0.061474   0.052812                                  
    3      0.052966   0.051033                                  
    4      0.053826   0.052875                                  
    5      0.052001   0.048456                                  
    6      0.056809   0.048939                                  
    7      0.058547   0.049012                                  
    8      0.055328   0.045812                                  
    9      0.053131   0.04595                                   
    10     0.053966   0.045393                                  
    11     0.051473   0.045612                                  
    12     0.047847   0.046206                                  
    13     0.051698   0.043556                                  
    14     0.048082   0.043449                                  
    15     0.043666   0

[0.041532446]

In [104]:
# cb[0].plot_lr()
# cb[0].plot_loss()

In [105]:
# it = iter(md.trn_dl)
# *xs,yts = next(it)
# t = model(*V(xs))

# xs[0].size(), yts.size()

In [106]:
# opt = optim.Adam(model.parameters(), 1e-2)
# fit(model, md, 2, opt, F.binary_cross_entropy)
# set_lrs(opt, 1e-3)
# fit(model, md, 3, opt, F.binary_cross_entropy)

## Predictions

In [107]:
preds = predict(model, test_dl)
preds.shape

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/module.py:357: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  result = self.forward(*input, **kwargs)
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  


(153164, 6)

## Prepare submission

In [108]:
subm_df = pd.read_csv("data/test.csv")

for i, col in enumerate(["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]):
    subm_df[col] = preds[:, i]

subm_df.head(2)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,"Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,",0.998924,0.333234,0.969354,0.054596,0.945446,0.271051
1,0000247867823ef7,"== From RfC == \n\n The title is fine as it is, IMO.",0.002046,0.000065,0.000780,0.000029,0.000780,0.000138


In [109]:
# if you want to write the submission file to disk, uncomment and run the below code
subm_df.drop(['comment_text'], axis=1).to_csv(f'{PATH}/submissions/lstm+conv_subm_20180319.csv', index=False)

## K-Fold Cross Validation

In [49]:
train_datafields = [("id", None), # we won't be needing the id, so we pass in None as the field
                    (txt_col, TEXT_fld),
                    ("toxic", LABEL_fld), ("severe_toxic", LABEL_fld), ("obscene", LABEL_fld),
                    ("threat", LABEL_fld), ("insult", LABEL_fld), ("identity_hate", LABEL_fld), ("none", None)]

# test
test_datafields = [("id", None), (txt_col, TEXT_fld)]

# define test dataset and iterator
test_ds = data.TabularDataset(f'{PATH}/test_ds.csv', format='csv', skip_header=True, fields=test_datafields)
test_iter = data.Iterator(test_ds, batch_size=batch_sizes[2], device=0, train=False, 
                          shuffle=False, sort=False, sort_within_batch=False, repeat=False)
test_dl = BatchWrapper(test_iter, txt_col, None)

# define FULL train dataset for building vocab
full_train_ds = data.TabularDataset(f'{PATH}/full_train_ds.csv', 
                                    format='csv', skip_header=True, fields=train_datafields)

TEXT_fld.build_vocab(full_train_ds, min_freq=min_freq, max_size=max_features, vectors=pretrained_vectors)

# cv
for i in range(n_folds):
    print('-' * 10)
    print(f'Fold {i} ....')
    
    # train/validation datsets
    train_ds, valid_ds = data.TabularDataset.splits(PATH, 
                                                    train=f'train_ds_{i}_of_{n_folds}.csv', 
                                                    validation=f'valid_ds_{i}_of_{n_folds}.csv',
                                                    format='csv', skip_header=True, fields=train_datafields)

    # train/validation iterators/dataloaders
    train_iter, val_iter = data.BucketIterator.splits(
        (train_ds, valid_ds), # we pass in the datasets we want the iterator to draw data from
        batch_sizes=(batch_sizes[0], batch_sizes[1]),
        device=0, # if you want to use the GPU, specify the GPU number here
        sort_key=lambda x: len(x.comment_text_cleaned), # the BucketIterator needs to be told what function it should use to group the data.
        sort_within_batch=False,
        repeat=False) # we pass repeat=False because we want to wrap this Iterator layer.
        
    train_dl = BatchWrapper(train_iter, txt_col, label_cols)
    valid_dl = BatchWrapper(val_iter, txt_col, label_cols)
        
    md = ModelData(PATH, trn_dl=train_dl, val_dl=valid_dl, test_dl=test_dl)
    
    model = SimpleGru(vocab_sz, emb_sz, n_rnn_hidden, n_rnn_layers, True, out_sz, bsz=batch_sizes[0],
                             linears=linears, linear_drops=linear_drops, use_bn=use_bn)
    
    model.cuda()
    
    lo = LayerOptimizer(optim.Adam, model, 1e-2, 1e-5)
    on_end = lambda sched, cycle: save_model(model, f'{PATH}/models/fit_1_cv{i}_cyc_{cycle}')
    cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2)] #, on_cycle_end=on_end)]
    fit(model, md, 2**4-1, lo.opt, F.binary_cross_entropy, callbacks=cb)
        
    on_end = lambda sched, cycle: save_model(model, f'{PATH}/models/fit_2_cv{i}_cyc_{cycle}')
    lo = LayerOptimizer(optim.Adam, model, 1e-3, 1e-5)
    cb = [CosAnneal(lo, (len(md.trn_dl) * 6))] #, on_cycle_end=on_end)]
    fit(model, md, 6, lo.opt, F.binary_cross_entropy, callbacks=cb)
        
    preds = predict(model, test_dl)
        
    subm_df = pd.read_csv("data/test.csv")
    for lbl_idx, col in enumerate(["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]):
        subm_df[col] = preds[:, lbl_idx]
        
    # if you want to write the submission file to disk, uncomment and run the below code
    subm_df.drop(['comment_text'], axis=1).to_csv(f'{PATH}/submissions/subm_20180317_cv_{i}.csv', index=False)



----------
Fold 0 ....


HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

  0%|          | 1/2244 [00:00<12:44,  2.93it/s, loss=0.593]

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/module.py:357: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  result = self.forward(*input, **kwargs)


/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration
  


...

epoch      trn_loss   val_loss                                  
    0      0.044418   0.044448  
    1      0.03985    0.043705                                  
    2      0.039774   0.044234                                  
    3      0.041095   0.043214                                  
    4      0.039615   0.043575                                  
    5      0.036174   0.043688                                  

----------
Fold 9 ....


HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss                                  
    0      0.048021   0.047052  
    1      0.056743   0.048223                                  
    2      0.044408   0.04363                                   
    3      0.057738   0.051385                                  
    4      0.051514   0.048031                                  
    5      0.048279   0.044213                                  
    6      0.045182   0.042637                                  
    7      0.064643   0.067792                                  
    8      0.056401   0.051193                                  
    9      0.057105   0.049551                                  
    10     0.052294   0.050228                                  
    11     0.05034    0.045985                                  
    12     0.047373   0.044812                                  
    13     0.044682   0.042055                                  
    14     0.040304   0.042011                           

HBox(children=(IntProgress(value=0, description='Epoch', max=6), HTML(value='')))

epoch      trn_loss   val_loss                                  
    0      0.042868   0.042617  
    1      0.040834   0.042033                                  
    2      0.041524   0.042581                                  
    3      0.038731   0.041987                                  
    4      0.0412     0.041493                                  
    5      0.037848   0.041736                                  



In [50]:
cv_dfs = []
for i in range(n_folds):
    df = pd.read_csv(f'{PATH}/submissions/subm_20180315_cv_{i}.csv')
    cv_dfs.append(df)
    
final_cv_df = pd.concat([ df for df in cv_dfs ])

display(len(final_cv_df))
display(final_cv_df.head(2))

1531640

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.996244,0.393182,0.972876,0.087797,0.935190,0.310491
1,0000247867823ef7,0.001040,0.000051,0.001035,0.000020,0.000553,0.000119


In [51]:
final_cv_df = final_cv_df.groupby(['id']).mean().reset_index()
final_cv_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.900280,0.375404,0.875346,0.087246,0.826584,0.242148
1,0000247867823ef7,0.004881,0.000273,0.002911,0.000026,0.001958,0.000360
2,00013b17ad220c46,0.030315,0.003812,0.022104,0.000283,0.014694,0.001177
3,00017563c3f7919a,0.017715,0.001505,0.010097,0.001165,0.010401,0.001894
4,00017695ad8997eb,0.012319,0.000488,0.004050,0.000300,0.002884,0.000992


In [52]:
final_cv_df.to_csv(f'{PATH}/submissions/subm_20180315_cv_final.csv', index=False)

Can repeat process above with other models/hyperparameters

In [124]:
train_datafields = [("id", None), # we won't be needing the id, so we pass in None as the field
                    (txt_col, TEXT_fld),
                    ("toxic", LABEL_fld), ("severe_toxic", LABEL_fld), ("obscene", LABEL_fld),
                    ("threat", LABEL_fld), ("insult", LABEL_fld), ("identity_hate", LABEL_fld), ("none", None)]

# test
test_datafields = [("id", None), (txt_col, TEXT_fld)]

# define test dataset and iterator
test_ds = data.TabularDataset(f'{PATH}/test_ds.csv', format='csv', skip_header=True, fields=test_datafields)
test_iter = data.Iterator(test_ds, batch_size=batch_sizes[2], device=0, train=False, 
                          shuffle=False, sort=False, sort_within_batch=False, repeat=False)
test_dl = BatchWrapper(test_iter, txt_col, None)

# define FULL train dataset for building vocab
full_train_ds = data.TabularDataset(f'{PATH}/full_train_ds.csv', 
                                    format='csv', skip_header=True, fields=train_datafields)

TEXT_fld.build_vocab(full_train_ds, min_freq=min_freq, max_size=max_features, vectors=pretrained_vectors)

# cv
for i in range(n_folds):
    print('-' * 10)
    print(f'Fold {i} ....')
    
    # train/validation datsets
    train_ds, valid_ds = data.TabularDataset.splits(PATH, 
                                                    train=f'train_ds_{i}_of_{n_folds}.csv', 
                                                    validation=f'valid_ds_{i}_of_{n_folds}.csv',
                                                    format='csv', skip_header=True, fields=train_datafields)

    # train/validation iterators/dataloaders
    train_iter, val_iter = data.BucketIterator.splits(
        (train_ds, valid_ds), # we pass in the datasets we want the iterator to draw data from
        batch_sizes=(batch_sizes[0], batch_sizes[1]),
        device=0, # if you want to use the GPU, specify the GPU number here
        sort_key=lambda x: len(x.comment_text_cleaned), # the BucketIterator needs to be told what function it should use to group the data.
        sort_within_batch=False,
        repeat=False) # we pass repeat=False because we want to wrap this Iterator layer.
        
    train_dl = BatchWrapper(train_iter, txt_col, label_cols)
    valid_dl = BatchWrapper(val_iter, txt_col, label_cols)
        
    md = ModelData(PATH, trn_dl=train_dl, val_dl=valid_dl, test_dl=test_dl)
    
    model = SimpleLstm(vocab_sz, emb_sz, n_rnn_hidden, n_rnn_layers, True, out_sz, bsz=batch_sizes[0],
                             linears=linears, linear_drops=linear_drops, use_bn=use_bn)
    
    model.cuda()
    
    lo = LayerOptimizer(optim.Adam, model, 1e-2, 1e-5)
    on_end = lambda sched, cycle: save_model(model, f'{PATH}/models/lstm_fit_1_cv{i}_cyc_{cycle}')
    cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2)] #, on_cycle_end=on_end)]
    fit(model, md, 2**4-1, lo.opt, F.binary_cross_entropy, callbacks=cb)
        
    on_end = lambda sched, cycle: save_model(model, f'{PATH}/models/lstm_fit_2_cv{i}_cyc_{cycle}')
    lo = LayerOptimizer(optim.Adam, model, 1e-3, 1e-5)
    cb = [CosAnneal(lo, (len(md.trn_dl) * 6))] #, on_cycle_end=on_end)]
    fit(model, md, 6, lo.opt, F.binary_cross_entropy, callbacks=cb)
        
    preds = predict(model, test_dl)
        
    subm_df = pd.read_csv("data/test.csv")
    for lbl_idx, col in enumerate(["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]):
        subm_df[col] = preds[:, lbl_idx]
        
    # if you want to write the submission file to disk, uncomment and run the below code
    subm_df.drop(['comment_text'], axis=1).to_csv(f'{PATH}/submissions/lstm_subm_20180317_cv_{i}.csv', index=False)



----------
Fold 0 ....


HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

  0%|          | 1/2244 [00:00<13:46,  2.71it/s, loss=0.69]     

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/module.py:357: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  result = self.forward(*input, **kwargs)


  3%|▎         | 60/2244 [00:04<03:01, 12.02it/s, loss=0.316]

...

epoch      trn_loss   val_loss                                  
    0      0.045647   0.044474  
    1      0.042817   0.044228                                  
    2      0.040915   0.045599                                  
    3      0.041809   0.043419                                  
    4      0.040474   0.043428                                  
    5      0.037941   0.043692                                  

----------
Fold 9 ....


HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss                                  
    0      0.049396   0.049165  
    1      0.05688    0.050416                                  
    2      0.047768   0.045109                                  
    3      0.058394   0.054652                                  
    4      0.051842   0.048766                                  
    5      0.04766    0.044876                                  
    6      0.046309   0.043491                                  
    7      0.059311   0.051079                                  
    8      0.056766   0.053325                                  
    9      0.060314   0.050524                                  
    10     0.052698   0.0494                                    
    11     0.050007   0.046307                                  
    12     0.048753   0.044859                                  
    13     0.046021   0.043258                                  
    14     0.041682   0.043055                           

HBox(children=(IntProgress(value=0, description='Epoch', max=6), HTML(value='')))

epoch      trn_loss   val_loss                                  
    0      0.046627   0.044108  
    1      0.044489   0.042747                                  
    2      0.044756   0.042827                                  
    3      0.043293   0.042333                                  
    4      0.042247   0.042378                                  
    5      0.037156   0.042477                                  



In [125]:
cv_dfs = []
for i in range(n_folds):
    df = pd.read_csv(f'{PATH}/submissions/lstm_subm_20180317_cv_{i}.csv')
    cv_dfs.append(df)
    
cv_dfs.append(pd.read_csv(f'{PATH}/submissions/lstm_subm_20180317.csv'))
    
final_cv_df = pd.concat([ df for df in cv_dfs ])

display(len(final_cv_df))
display(final_cv_df.head(2))

1684804

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.998808,0.538410,0.986663,0.125191,0.959094,0.386088
1,0000247867823ef7,0.001600,0.000077,0.000564,0.000135,0.000491,0.000188


In [126]:
final_cv_df = final_cv_df.groupby(['id']).mean().reset_index()
final_cv_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.996908,0.410974,0.975145,0.092718,0.940464,0.253616
1,0000247867823ef7,0.002657,0.000136,0.001047,0.000120,0.000590,0.000182
2,00013b17ad220c46,0.035074,0.001136,0.012181,0.000598,0.005512,0.001199
3,00017563c3f7919a,0.001018,0.000057,0.000456,0.000067,0.000361,0.000068
4,00017695ad8997eb,0.007170,0.000335,0.002264,0.000550,0.001165,0.000365


In [127]:
final_cv_df.to_csv(f'{PATH}/submissions/lstm_subm_20180317_cv_final.csv', index=False)

Ensemble a bunch of different model predictions into the grand daddy of all submissions

In [73]:
cv_dfs = []
for i in range(n_folds):
    df = pd.read_csv(f'{PATH}/submissions/lstm_subm_20180315_cv_{i}.csv')
    df2 = pd.read_csv(f'{PATH}/submissions/subm_20180315_cv_{i}.csv')
    cv_dfs.append(df)
    cv_dfs.append(df2)
    
final_cv_df = pd.concat([ df for df in cv_dfs ])

display(len(final_cv_df))
display(final_cv_df.head(2))

3063280

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.996635,0.486019,0.980172,0.184098,0.950054,0.295634
1,0000247867823ef7,0.003302,0.000202,0.001431,0.000136,0.000833,0.000203


In [74]:
final_cv_df = final_cv_df.groupby(['id']).mean().reset_index()
final_cv_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.948321,0.389054,0.923985,0.091080,0.882614,0.260359
1,0000247867823ef7,0.004504,0.000229,0.002165,0.000083,0.001435,0.000301
2,00013b17ad220c46,0.030084,0.002457,0.015929,0.000577,0.009565,0.001025
3,00017563c3f7919a,0.009609,0.000786,0.005316,0.000637,0.005454,0.001003
4,00017695ad8997eb,0.011514,0.000454,0.003311,0.000428,0.002181,0.000706


In [75]:
final_cv_df.to_csv(f'{PATH}/submissions/subm_gru+lstm_final.csv', index=False)